# DataFrame

The following codes are demos only. It's **NOT for production** due to system security concerns, please **DO NOT** use it directly in production.

It is recommended to use [jupyter](https://jupyter.org/) to run this tutorial.

Secretflow provides federated data encapsulation in the form of DataFrame. DataFrame is composed of data blocks of multiple parties and supports horizontal or vertical partitioned data.

<img alt="dataframe.png" src="resource/dataframe.png" width="600">

Currently secretflow.DataFrame provides a subset of pandas operations, which are basically the same as pandas. During the calculation process, the original data is kept in the data holder and will not go out of the domain.



The following will demonstrate how to use a DataFrame.

## Preparation

Initialize secretflow and create two parties alice and bob.

In [16]:
import secretflow as sf

# In case you have a running secretflow runtime already.
sf.shutdown()

sf.init(['alice', 'bob'])
alice = sf.PYU('alice')
bob = sf.PYU('bob')

## Data preparation

Here we use [iris](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_iris.html) as example data.

In [2]:
import pandas as pd
from sklearn.datasets import load_iris

iris = load_iris(as_frame=True)
data = pd.concat([iris.data, iris.target], axis=1)
data

sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
0                  5.1               3.5                1.4               0.2   
1                  4.9               3.0                1.4               0.2   
2                  4.7               3.2                1.3               0.2   
3                  4.6               3.1                1.5               0.2   
4                  5.0               3.6                1.4               0.2   
..                 ...               ...                ...               ...   
145                6.7               3.0                5.2               2.3   
146                6.3               2.5                5.0               1.9   
147                6.5               3.0                5.2               2.0   
148                6.2               3.4                5.4               2.3   
149                5.9               3.0                5.1               1.8   

     target  
0         0  
1         0  
2         0  
3         0  
4         0  
..      ...  
145       2  
146       2  
147       2  
148       2  
149       2  

[150 rows x 5 columns]

We divide the data according to horizontal (the same features, each holds some samples) and vertical mode (each holds some features) to facilitate subsequent display.

In [3]:
# Horizontal partitioning.
h_alice, h_bob = data.iloc[:70, :], data.iloc[70:, :]

# Save to temporary files.
import tempfile

_, h_alice_path = tempfile.mkstemp()
_, h_bob_path = tempfile.mkstemp()
h_alice.to_csv(h_alice_path, index=False)
h_bob.to_csv(h_bob_path, index=False)

In [5]:
h_alice

sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
0                 5.1               3.5                1.4               0.2   
1                 4.9               3.0                1.4               0.2   
2                 4.7               3.2                1.3               0.2   
3                 4.6               3.1                1.5               0.2   
4                 5.0               3.6                1.4               0.2   
..                ...               ...                ...               ...   
65                6.7               3.1                4.4               1.4   
66                5.6               3.0                4.5               1.5   
67                5.8               2.7                4.1               1.0   
68                6.2               2.2                4.5               1.5   
69                5.6               2.5                3.9               1.1   

    target  
0        0  
1        0  
2        0  
3        0  
4        0  
..     ...  
65       1  
66       1  
67       1  
68       1  
69       1  

[70 rows x 5 columns]

In [6]:
h_bob

sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
70                 5.9               3.2                4.8               1.8   
71                 6.1               2.8                4.0               1.3   
72                 6.3               2.5                4.9               1.5   
73                 6.1               2.8                4.7               1.2   
74                 6.4               2.9                4.3               1.3   
..                 ...               ...                ...               ...   
145                6.7               3.0                5.2               2.3   
146                6.3               2.5                5.0               1.9   
147                6.5               3.0                5.2               2.0   
148                6.2               3.4                5.4               2.3   
149                5.9               3.0                5.1               1.8   

     target  
70        1  
71        1  
72        1  
73        1  
74        1  
..      ...  
145       2  
146       2  
147       2  
148       2  
149       2  

[80 rows x 5 columns]

In [6]:
# Vertical partitioning.
v_alice, v_bob = data.iloc[:, :2], data.iloc[:, 2:]

# Save to temporary files.
_, v_alice_path = tempfile.mkstemp()
_, v_bob_path = tempfile.mkstemp()
v_alice.to_csv(v_alice_path, index=False)
v_bob.to_csv(v_bob_path, index=False)

In [8]:
v_alice

sepal length (cm)  sepal width (cm)
0                  5.1               3.5
1                  4.9               3.0
2                  4.7               3.2
3                  4.6               3.1
4                  5.0               3.6
..                 ...               ...
145                6.7               3.0
146                6.3               2.5
147                6.5               3.0
148                6.2               3.4
149                5.9               3.0

[150 rows x 2 columns]

In [9]:
v_bob

petal length (cm)  petal width (cm)  target
0                  1.4               0.2       0
1                  1.4               0.2       0
2                  1.3               0.2       0
3                  1.5               0.2       0
4                  1.4               0.2       0
..                 ...               ...     ...
145                5.2               2.3       2
146                5.0               1.9       2
147                5.2               2.0       2
148                5.4               2.3       2
149                5.1               1.8       2

[150 rows x 3 columns]

## Creation

### Horitontal DataFrame

Create a DataFrame consisting of horizontally partitioned data.

> 💡 The original data is still stored locally in the data holder and is not transmitted out of the domain.

Here, as a simple show case, we choose plaintext aggregation and comparison. It's not recommend to use it in production. You can refer to [Security Aggregation](./Secure_aggregation.ipynb) to learn more about security aggregation solutions and implement appropriate security policies according to your needs.

In [11]:
from secretflow.data.horizontal import read_csv as h_read_csv
from secretflow.security.aggregation import SecureAggregator
from secretflow.security.compare import SPUComparator

# The aggregator and comparator are respectively used to aggregate 
# or compare data in subsequent data analysis operations.
aggr = SecureAggregator(device=alice, participants=[alice, bob])
spu = sf.SPU(sf.utils.testing.cluster_def(parties=['alice', 'bob']))
comp = SPUComparator(spu)
hdf = h_read_csv({alice: h_alice_path, bob: h_bob_path}, 
                 aggregator=aggr, 
                 comparator=comp)

### Vertical DataFrame

Create a DataFrame consisting of vertically partitioned data.

> 💡 The original data is still stored locally in the data holder and is not transmitted out of the domain.

In [7]:
from secretflow.data.vertical import read_csv as v_read_csv

vdf = v_read_csv({alice: v_alice_path, bob: v_bob_path})

## Data analysis

For data privacy protection purposes, DataFrame does not allow the view of raw data. DataFrame provides an interface similar to pandas for users to analyze data. These interfaces usually support both horizontal and vertical partitioned data.

> 💡 During the following operations, the original data of the DataFrame is still stored locally on the node and is not transmitted out of the domain.

In [8]:
hdf.columns

Index(['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)',
       'petal width (cm)', 'target'],
      dtype='object')

In [9]:
vdf.columns

Index(['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)',
       'petal width (cm)', 'target'],
      dtype='object')

Get the minimum value, you can see that it is consistent with the original data.

In [14]:
print('Horizontal df:\n', hdf.min())
print('\nVertical df:\n', vdf.min())
print('\nPandas:\n', data.min())

Horizontal df:
 sepal length (cm)    4.3
sepal width (cm)     2.0
petal length (cm)    1.0
petal width (cm)     0.1
target               0.0
dtype: float64

Vertical df:
 sepal length (cm)    4.3
sepal width (cm)     2.0
petal length (cm)    1.0
petal width (cm)     0.1
target               0.0
dtype: float64

Pandas:
 sepal length (cm)    4.3
sepal width (cm)     2.0
petal length (cm)    1.0
petal width (cm)     0.1
target               0.0
dtype: float64


You can also view information such as maximum value, mean value, and quantity.

In [12]:
hdf.max()

sepal length (cm)    7.9
sepal width (cm)     4.4
petal length (cm)    6.9
petal width (cm)     2.5
target               2.0
dtype: float64

In [13]:
vdf.max()

sepal length (cm)    7.9
sepal width (cm)     4.4
petal length (cm)    6.9
petal width (cm)     2.5
target               2.0
dtype: float64

In [14]:
hdf.mean(numeric_only=True)

sepal length (cm)    1.168667
sepal width (cm)     0.611467
petal length (cm)    0.751600
petal width (cm)     0.239867
target               0.200000
dtype: float64

In [15]:
vdf.mean(numeric_only=True)

sepal length (cm)    5.843333
sepal width (cm)     3.057333
petal length (cm)    3.758000
petal width (cm)     1.199333
target               1.000000
dtype: float64

In [19]:
hdf.count()

sepal length (cm)    150
sepal width (cm)     150
petal length (cm)    150
petal width (cm)     150
target               150
dtype: int64

In [20]:
vdf.count()

sepal length (cm)    150
sepal width (cm)     150
petal length (cm)    150
petal width (cm)     150
target               150
dtype: int64

### Selection

Get partial columns.

In [21]:
hdf_part = hdf[['sepal length (cm)', 'target']]
hdf_part.mean(numeric_only=True)

sepal length (cm)    5.843333
target               1.000000
dtype: float64

In [22]:
vdf_part = vdf[['sepal width (cm)', 'target']]
vdf_part.mean(numeric_only=True)

sepal width (cm)    3.057333
target              1.000000
dtype: float64

### Modification

Horizontal DataFrame

In [23]:
hdf_copy = hdf.copy()
print('Min of target: ', hdf_copy['target'].min()[0])
print('Max of target: ', hdf_copy['target'].max()[0])

Min of target:  0.0
Max of target:  2.0


In [24]:
# Set target to 1。
hdf_copy['target'] = 1

# You can see that the value of target has become 1.
print('Min of target: ', hdf_copy['target'].min()[0])
print('Max of target: ', hdf_copy['target'].max()[0])

Min of target:  1.0
Max of target:  1.0


Vertical DataFrame.

In [25]:
vdf_copy = vdf.copy()
print('Min of sepal width (cm): ', vdf_copy['sepal width (cm)'].min()[0])
print('Max of sepal width (cm): ', vdf_copy['sepal width (cm)'].max()[0])

Min of sepal width (cm):  2.0
Max of sepal width (cm):  4.4


In [26]:
# Set sepal width (cm) to 20。
vdf_copy['sepal width (cm)'] = 20

# You can see that the value of sepal width (cm) has become 20.
print('Min of sepal width (cm): ', vdf_copy['sepal width (cm)'].min()[0])
print('Max of sepal width (cm): ', vdf_copy['sepal width (cm)'].max()[0])

Min of sepal width (cm):  20
Max of sepal width (cm):  20


## Ending

In [26]:
# Clean up temporary files

import os

try:
    os.remove(h_alice_path)
    os.remove(h_bob_path)
except OSError:
    pass

try:
    os.remove(v_alice_path)
    os.remove(v_bob_path)
except OSError:
    pass